In [1]:
def call_indicators(key):
    '''This function calls the Technical Indicators class from the Alpha Vantage package.
    Input: 
    key: Alpha Vantage API Key
    '''
    from alpha_vantage.techindicators import TechIndicators
    return TechIndicators(key, output_format='pandas')

In [2]:
def call_timeseries(key):
    '''This function calls the Time Series class from the Alpha Vantage package.
    Input: 
    key: Alpha Vantage API Key
    '''
    from alpha_vantage.timeseries import TimeSeries
    return TimeSeries(key, output_format='pandas')

In [3]:
import numpy as np

In [4]:
def update_xs(indicator,symbol, key):
    '''Returns a JSON Dictionary containing the last data points for a specified indicator on a specific stock.
    Numpy must be imported before use.
    Inputs:
    - indicator: technical indicator, chosen from **LIST HERE**
    - symbol of the stock chosen, eg. Microsoft
    - key: Alpha Vantage API Key
    Returns: numpy array'''
    
    #Call the Technical Indicators class from Alpha Vantage
    techin = call_indicators(key)
    #This should maybe be done with more data points.
    if indicator == 'MOM':
        data, meta_data = techin.get_mom(symbol=symbol, interval='daily')
        large_MOM = np.array(data['MOM'])
        small_MOM = large_MOM[-1:-8:-1] #Returns MOM values for the last seven working days
        return small_MOM

    

In [15]:
def array_xs(symbol, key):
    '''Returns a JSON Dictionary containing the last data points for a specified indicator on a specific stock.
    Numpy must be imported before use.
    Inputs:
    - indicator: technical indicator, chosen from **LIST HERE**
    - symbol of the stock chosen, eg. Microsoft
    - key: Alpha Vantage API Key
    Returns: numpy array'''
    
    #Call the Technical Indicators class from Alpha Vantage
    techin = call_indicators(key)
    
    mom_data, meta_mom = techin.get_mom(symbol=symbol, interval='daily')
    large_MOM = np.array(mom_data['MOM'])
    
    sma_data, meta_sma = techin.get_sma(symbol, interval='daily', time_period=200, series_type='close')
    large_SMA = np.array(sma_data['SMA'])
    
    x = np.concatenate((large_MOM, large_SMA), axis=0)
    
    rsi_data , meta_rsi = techin.get_rsi(symbol, interval='daily', time_period=200, series_type='close')
    large_rsi = np.array(rsi_data['RSI'])
    
    x = np.concatenate((x, large_rsi), axis=0)
    
    return x

In [11]:
def label_ys(symbol, key):
    '''Returns a vector y with enteries labelled 1 or 0 based on whether the percentage increase has been
    met over a specific time period.
    Numpy must be imported before use.'''
    
    #Call the Time Series class from Alpha Vantage
    ts = call_timeseries(key)
    
    data, meta_data = ts.get_daily(symbol, outputsize='full')
    
    data_array = np.array(data['4. close'])
    smaller_array = data_array[-1:-200:-1]
    
    y = np.zeros(200-7)
    for i in range(7, 199):
        if smaller_array[i-7] >= 1.05*smaller_array[i]:
            y[i-7] = 1
    
    return y